# Loneliness and Deprivation

## Read in IMD and loneliness index data for England

In [1]:
import pandas as pd

In [20]:
imd = pd.read_csv('england/imd2015eng.csv')

In [21]:
imd.columns

Index(['LSOA code (2011)', 'LSOA name (2011)',
       'Local Authority District code (2013)',
       'Local Authority District name (2013)',
       'Index of Multiple Deprivation (IMD) Score',
       'Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived)',
       'Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs)',
       'Income Score (rate)', 'Income Rank (where 1 is most deprived)',
       'Income Decile (where 1 is most deprived 10% of LSOAs)',
       'Employment Score (rate)', 'Employment Rank (where 1 is most deprived)',
       'Employment Decile (where 1 is most deprived 10% of LSOAs)',
       'Education, Skills and Training Score',
       'Education, Skills and Training Rank (where 1 is most deprived)',
       'Education, Skills and Training Decile (where 1 is most deprived 10% of LSOAs)',
       'Health Deprivation and Disability Score',
       'Health Deprivation and Disability Rank (where 1 is most deprived)',
       'Health Dep

In [22]:
loneliness = pd.read_csv('england/final_data.csv')

In [23]:
loneliness

,Unnamed: 0,pcstrip,Year,NUMBER_OF_PATIENTS,SHA,PCT,oseast1m,osnrth1m,lsoa11,msoa11,...,alzheimers_zscore,blood pressure_zscore,hypertension_zscore,diabeties_zscore,cardiovascular disease_zscore,insomnia_zscore,addiction_zscore,social anxiety_zscore,loneliness_zscore,loneills
0,0,AL100BS,2015,9513.00,Q58,06K,522443.0,208996.0,E01023927,E02004990,...,-1.115736,-0.213050,-0.168954,-0.645499,-0.610262,-0.564285,-0.074779,-0.201186,-1.022917,-2.621486
1,1,AL100NL,2015,12392.25,Q58,06K,522442.0,208808.0,E01023920,E02004991,...,-1.337870,0.401903,0.375216,-0.082910,0.017223,-0.556142,-0.101102,-0.316347,-0.017028,-2.066433
2,2,AL108HP,2015,9585.75,Q58,06K,522445.0,208444.0,E01023922,E02004991,...,-1.199077,0.854120,0.906778,0.122991,0.942274,-1.073965,-0.101102,-0.257693,0.604950,-2.167174
3,3,AL13HD,2015,19392.00,Q58,06N,515081.0,207765.0,E01023676,E02004934,...,-1.134115,-0.420169,-0.439357,-0.448000,-0.688693,-0.817193,-0.101102,-0.059839,-0.990541,-2.626996
4,4,AL13JB,2015,18274.00,Q58,06N,514829.0,207213.0,E01023727,E02004935,...,-1.028534,-0.348092,-0.295509,-0.237027,-0.926088,0.230290,-0.063253,-0.535921,-0.704620,-1.604260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7063,7063,YO71LU,2015,8236.50,Q50,03D,442965.0,481974.0,E01027630,E02005757,...,0.174461,1.047097,1.019164,-0.649497,-0.702000,-1.110963,-0.101102,0.893232,-0.391322,0.179598
7064,7064,YO73RP,2015,2925.25,Q50,03D,440259.0,475953.0,E01027635,E02005758,...,-0.375846,1.007852,0.956097,-0.181015,0.993619,-1.178251,-0.101102,-0.460783,0.633195,-1.555965
7065,7065,YO84BL,2015,11071.75,Q50,03Q,461055.0,432478.0,E01027908,E02005813,...,-0.206876,-1.010645,-1.026337,-0.409296,-0.639128,-1.051432,-0.094154,-0.808738,-1.036607,-2.750970
7066,7066,YO84QH,2015,16039.50,Q50,03Q,461500.0,432020.0,E01027909,E02005813,...,0.259496,-0.732146,-0.761066,-0.843255,-0.311799,-0.921579,-0.017489,-0.591003,-0.586766,-1.266272


In [6]:
imd.shape

(32844, 57)

In [7]:
loneliness.shape

(7068, 35)

In [24]:
loneliness.columns

Index(['Unnamed: 0', 'pcstrip', 'Year', 'NUMBER_OF_PATIENTS', 'SHA', 'PCT',
       'oseast1m', 'osnrth1m', 'lsoa11', 'msoa11', 'ru11ind', 'rgn', 'laua',
       'imd', 'depression_perc', 'alzheimers_perc', 'blood pressure_perc',
       'hypertension_perc', 'diabeties_perc', 'cardiovascular disease_perc',
       'insomnia_perc', 'addiction_perc', 'social anxiety_perc',
       'loneliness_perc', 'depression_zscore', 'alzheimers_zscore',
       'blood pressure_zscore', 'hypertension_zscore', 'diabeties_zscore',
       'cardiovascular disease_zscore', 'insomnia_zscore', 'addiction_zscore',
       'social anxiety_zscore', 'loneliness_zscore', 'loneills'],
      dtype='object')

In [25]:
imd.columns

Index(['LSOA code (2011)', 'LSOA name (2011)',
       'Local Authority District code (2013)',
       'Local Authority District name (2013)',
       'Index of Multiple Deprivation (IMD) Score',
       'Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived)',
       'Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs)',
       'Income Score (rate)', 'Income Rank (where 1 is most deprived)',
       'Income Decile (where 1 is most deprived 10% of LSOAs)',
       'Employment Score (rate)', 'Employment Rank (where 1 is most deprived)',
       'Employment Decile (where 1 is most deprived 10% of LSOAs)',
       'Education, Skills and Training Score',
       'Education, Skills and Training Rank (where 1 is most deprived)',
       'Education, Skills and Training Decile (where 1 is most deprived 10% of LSOAs)',
       'Health Deprivation and Disability Score',
       'Health Deprivation and Disability Rank (where 1 is most deprived)',
       'Health Dep

IMD data is a single snapshot for each LSOA. Loneliness in theory has entries for each year for each postcode but in practice is limited to a single year. Will need to produce an aggregate view for each LSOA to fuse with the IMD data

In [10]:
loneliness['Year'].unique()

array([2015])

In [11]:
agg_cols = {'loneills': 'mean'}

# for key in [
#     'depression_perc', 'alzheimers_perc', 'blood pressure_perc', 'hypertension_perc', 'diabeties_perc', 'cardiovascular disease_perc', 'insomnia_perc', 'addiction_perc',
#     'social anxiety_perc', 'loneliness_perc'
# ]:
#     agg_cols[key] = 'mean'

# for key in ['NUMBER_OF_PATIENTS']:
#     agg_cols[key] = 'sum'

for key in ['msoa11']:
    agg_cols[key] = 'first'

loneliness_per_lsoa11 = loneliness.groupby(['lsoa11', 'Year'], as_index=False).agg(agg_cols)

In [12]:
loneliness_imd = pd.merge(loneliness_per_lsoa11, imd, how='left', left_on=['lsoa11'], right_on=['LSOA code (2011)'])

In [13]:
loneliness_imd.shape

(6271, 61)

## rename the columns

In [14]:
loneliness_imd.columns # look at these horrible names

Index(['lsoa11', 'Year', 'loneills', 'msoa11', 'LSOA code (2011)',
       'LSOA name (2011)', 'Local Authority District code (2013)',
       'Local Authority District name (2013)',
       'Index of Multiple Deprivation (IMD) Score',
       'Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived)',
       'Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs)',
       'Income Score (rate)', 'Income Rank (where 1 is most deprived)',
       'Income Decile (where 1 is most deprived 10% of LSOAs)',
       'Employment Score (rate)', 'Employment Rank (where 1 is most deprived)',
       'Employment Decile (where 1 is most deprived 10% of LSOAs)',
       'Education, Skills and Training Score',
       'Education, Skills and Training Rank (where 1 is most deprived)',
       'Education, Skills and Training Decile (where 1 is most deprived 10% of LSOAs)',
       'Health Deprivation and Disability Score',
       'Health Deprivation and Disability Rank (where 

In [15]:
# here is where I rename those columns

loneliness_imd = loneliness_imd.rename(columns={'Outdoors Sub-domain Score':"outdoor_score",
                               'Year': 'year',
                               'LSOA code (2011)': 'lsoa_code',
                               'LSOA name (2011)': 'lsoa_name',
                               'Local Authority District code (2013)': 'district_code',
                               'Local Authority District name (2013)': 'district_name',
                               'Index of Multiple Deprivation (IMD) Score': 'imd_score',
                               'Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived)': 'imd_rank',
                               'Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs)': 'imd_decile',
                               'Income Score (rate)': 'income_score',
                            'Income Rank (where 1 is most deprived)': 'income_rank',
       'Income Decile (where 1 is most deprived 10% of LSOAs)': 'income_decile',
       'Employment Score (rate)': 'employment_rate', 
                               'Employment Rank (where 1 is most deprived)': 'employment_score',
       'Employment Decile (where 1 is most deprived 10% of LSOAs)': 'employment_decile',
       'Education, Skills and Training Score': 'education_score',
       'Education, Skills and Training Rank (where 1 is most deprived)': 'education_rank',
       'Education, Skills and Training Decile (where 1 is most deprived 10% of LSOAs)': 'education_decile',
       'Health Deprivation and Disability Score': 'health_score',
       'Health Deprivation and Disability Rank (where 1 is most deprived)': 'health_rank',
       'Health Deprivation and Disability Decile (where 1 is most deprived 10% of LSOAs)': 'health_decile',
       'Crime Score': 'crime_score',
                               'Crime Rank (where 1 is most deprived)': 'crime_rank',
       'Crime Decile (where 1 is most deprived 10% of LSOAs)': 'crime_decile',
       'Barriers to Housing and Services Score': 'housing_score',
       'Barriers to Housing and Services Rank (where 1 is most deprived)': 'housing_rank',
       'Barriers to Housing and Services Decile (where 1 is most deprived 10% of LSOAs)': 'housing_decile',
       'Living Environment Score': 'living_score',
       'Living Environment Rank (where 1 is most deprived)': 'living_rank',
       'Living Environment Decile (where 1 is most deprived 10% of LSOAs)': 'living_decile',
       'Income Deprivation Affecting Children Index (IDACI) Score (rate)':'poor_children_score',
       'Income Deprivation Affecting Children Index (IDACI) Rank (where 1 is most deprived)': 'poor_children_rank',
       'Income Deprivation Affecting Children Index (IDACI) Decile (where 1 is most deprived 10% of LSOAs)': 'poor_children_decile',
       'Income Deprivation Affecting Older People (IDAOPI) Score (rate)': 'poor_older_score',
       'Income Deprivation Affecting Older People (IDAOPI) Rank (where 1 is most deprived)': 'poor_older_rank',
       'Income Deprivation Affecting Older People (IDAOPI) Decile (where 1 is most deprived 10% of LSOAs)': 'poor_older_decile',
       'Children and Young People Sub-domain Score': 'young_sub_score',
       'Children and Young People Sub-domain Rank (where 1 is most deprived)': 'young_sub_rank',
       'Children and Young People Sub-domain Decile (where 1 is most deprived 10% of LSOAs)':'young_sub_decile',
       'Adult Skills Sub-domain Score': 'adult_skills_sub_score',
       'Adult Skills Sub-domain Rank (where 1 is most deprived)': 'adult_skills_sub_rank',
       'Adult Skills Sub-domain Decile (where 1 is most deprived 10% of LSOAs)': 'adult_skills_sub_decile',
       'Geographical Barriers Sub-domain Score': 'geog_barriers_sub_score',
       'Geographical Barriers Sub-domain Rank (where 1 is most deprived)': 'geog_barriers_sub_rank',
       'Geographical Barriers Sub-domain Decile (where 1 is most deprived 10% of LSOAs)': 'geog_barriers_sub_decile',
       'Wider Barriers Sub-domain Score': 'wider_barriers_sub_score',
       'Wider Barriers Sub-domain Rank (where 1 is most deprived)': 'wider_barriers_sub_rank',
       'Wider Barriers Sub-domain Decile (where 1 is most deprived 10% of LSOAs)': 'wider_barriers_sub_decile',
       'Indoors Sub-domain Score': 'indoor_sub_score',
       'Indoors Sub-domain Rank (where 1 is most deprived)': 'indoor_sub_rank',
       'Indoors Sub-domain Decile (where 1 is most deprived 10% of LSOAs)': 'indoor_sub_decile',
       'Outdoors Sub-domain Score': 'outdoor_sub_score',
       'Outdoors Sub-domain Rank (where 1 is most deprived)': 'outdoor_sub_rank',
       'Outdoors Sub-domain Decile (where 1 is most deprived 10% of LSOAs)': 'outdoor_sub_decile',
       'Total population: mid 2012 (excluding prisoners)':'total_population',
       'Dependent Children aged 0-15: mid 2012 (excluding prisoners)':'aged_under_15_population',
       'Population aged 16-59: mid 2012 (excluding prisoners)': 'aged_16-59_population',
       'Older population aged 60 and over: mid 2012 (excluding prisoners)': 'aged_over_60_population',
       'Working age population 18-59/64: for use with Employment Deprivation Domain (excluding prisoners) ': 'working_age_population'})

In [16]:
loneliness_imd.columns

Index(['lsoa11', 'year', 'loneills', 'msoa11', 'lsoa_code', 'lsoa_name',
       'district_code', 'district_name', 'imd_score', 'imd_rank', 'imd_decile',
       'income_score', 'income_rank', 'income_decile', 'employment_rate',
       'employment_score', 'employment_decile', 'education_score',
       'education_rank', 'education_decile', 'health_score', 'health_rank',
       'health_decile', 'crime_score', 'crime_rank', 'crime_decile',
       'housing_score', 'housing_rank', 'housing_decile', 'living_score',
       'living_rank', 'living_decile', 'poor_children_score',
       'poor_children_rank', 'poor_children_decile', 'poor_older_score',
       'poor_older_rank', 'poor_older_decile', 'young_sub_score',
       'young_sub_rank', 'young_sub_decile', 'adult_skills_sub_score',
       'adult_skills_sub_rank', 'adult_skills_sub_decile',
       'geog_barriers_sub_score', 'geog_barriers_sub_rank',
       'geog_barriers_sub_decile', 'wider_barriers_sub_score',
       'wider_barriers_sub_rank',

In [32]:
loneliness_imd.to_csv('england/loneliness_imd.csv', index=False)